<a href="https://colab.research.google.com/github/akashmodak97/Genre-Detection-Of-Bengali-Songs-Based-On-Audio-Data/blob/master/Music_Genre_Classification_using_LSTM_based_on_audio_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Created By Akash Modak ##
## Date 25/06/2020 Time 21:05 ##
import numpy as np 
import pandas as pd
import csv
import io
from google.colab import files
# uploaded = files.upload()
# d = pd.read_csv(io.BytesIO(uploaded['combined1.csv']))

# features = []
# names = []
# print(data)
# with open('final.csv','rt')as f:
#   data = csv.reader(f)
#   for row in data:
#     features.append(np.array(row[0:]))
  
# # print(uploaded['combined1.csv'])
# # print(d)
# features = np.array(features)
# print(features)
df_train = pd.read_csv('combined2.csv')
print('train set: {0}'.format(df_train.shape))
df_train.head(10)

train set: (1373, 56)


,song_id,tempo,total_beats,average_beats,chroma_stft_mean,chroma_stft_std,chroma_stft_var,chroma_cq_mean,chroma_cq_std,chroma_cq_var,chroma_cens_mean,chroma_cens_std,chroma_cens_var,melspectrogram_mean,melspectrogram_std,melspectrogram_var,mfcc_mean,mfcc_std,mfcc_var,mfcc_delta_mean,mfcc_delta_std,mfcc_delta_var,rmse_mean,rmse_std,rmse_var,cent_mean,cent_std,cent_var,spec_bw_mean,spec_bw_std,spec_bw_var,contrast_mean,contrast_std,contrast_var,rolloff_mean,rolloff_std,rolloff_var,poly_mean,poly_std,poly_var,tonnetz_mean,tonnetz_std,tonnetz_var,zcr_mean,zcr_std,zcr_var,harm_mean,harm_std,harm_var,perc_mean,perc_std,perc_var,frame_mean,frame_std,frame_var,label
0,1,117.453835,143490,1258.684211,0.265287,0.307550,0.094587,0.476079,0.268147,0.071903,0.256190,0.133041,0.017700,19.448763,132.005372,17425.418170,3.232830,32.172203,1035.050631,0.005313,2.248681,5.056564,0.278877,0.098115,0.009627,1816.690740,443.095197,196333.3535,2170.739580,273.765858,74947.74502,22.411474,6.577048,43.257561,3694.565159,1136.062437,1.290638e+06,3.002472,3.315041,10.989497,-0.002156,0.076395,0.005836,0.071314,0.024713,0.000611,6.000000e-05,0.246668,0.060845,0.000306,0.094004,0.008837,1.765878,1.092411,1.193362,3
1,2,129.199219,164643,1276.302326,0.240892,0.301678,0.091010,0.366112,0.285707,0.081629,0.226792,0.178602,0.031899,5.013715,36.131561,1305.489722,-7.652302,58.106209,3376.331495,0.009265,1.959212,3.838512,0.132475,0.064600,0.004173,1473.438015,705.663562,497961.0631,1655.685724,498.834614,248835.97250,25.016916,6.683738,44.672352,2945.661355,1442.399044,2.080515e+06,1.276907,1.537728,2.364606,-0.013783,0.104215,0.010861,0.069444,0.046261,0.002140,-4.480000e-06,0.137462,0.018896,-0.000027,0.027201,0.000740,8.467156,4.649220,21.615245,3
2,3,151.999081,199193,1293.461039,0.275192,0.300286,0.090171,0.414688,0.282245,0.079662,0.237777,0.163693,0.026795,4.973484,35.085509,1230.992971,-7.458312,45.654727,2084.354112,0.007591,2.395649,5.739136,0.136274,0.064940,0.004217,1852.490285,407.523515,166075.4154,1895.885455,312.704488,97784.09662,24.167576,7.026549,49.372389,3805.789487,919.714851,8.458754e+05,1.492162,1.765966,3.118638,-0.006023,0.092215,0.008504,0.088848,0.038953,0.001517,3.660000e-07,0.135930,0.018477,-0.000004,0.037525,0.001408,2.594830,1.129929,1.276739,3
3,4,151.999081,161883,1217.165414,0.248626,0.297679,0.088613,0.371241,0.289661,0.083904,0.216809,0.190597,0.036327,0.437058,4.863900,23.657528,-13.042703,79.185898,6270.406471,0.006632,1.680400,2.823745,0.041848,0.026085,0.000680,1144.678307,427.171290,182475.3108,1523.492834,273.218664,74648.43815,26.073470,5.826027,33.942588,2333.598389,851.338219,7.247768e+05,0.395086,0.583674,0.340675,0.022566,0.103551,0.010723,0.039310,0.033671,0.001134,-2.177326e-02,0.039642,0.001572,-0.000003,0.012040,0.000145,8.168780,3.537997,12.517425,3
4,5,161.499023,210099,1296.907407,0.281117,0.306030,0.093654,0.399010,0.281916,0.079477,0.241224,0.158569,0.025144,3.054194,16.763283,281.007651,-8.703200,54.125674,2929.588607,0.006666,2.623608,6.883319,0.108530,0.040449,0.001636,1549.812453,371.013724,137651.1831,1862.057693,344.853866,118924.18900,25.644132,9.582786,91.829782,3055.902355,913.809451,8.350477e+05,1.198203,1.431780,2.049994,0.003348,0.094996,0.009024,0.065280,0.023577,0.000556,-9.460000e-07,0.096812,0.009372,-0.000028,0.039396,0.001552,2.987247,1.572752,2.473548,3
5,6,143.554688,183554,1274.680556,0.262019,0.316322,0.100060,0.383035,0.289213,0.083644,0.226645,0.178789,0.031965,0.878722,6.509083,42.368163,-7.695001,60.273554,3632.901291,0.007846,2.389611,5.710242,0.061965,0.022247,0.000495,1557.534661,391.023980,152899.7530,1890.830789,344.951737,118991.70060,23.624107,6.285628,39.509118,3223.113844,860.966826,7.412639e+05,0.642782,0.739266,0.546513,0.007231,0.091871,0.008440,0.059199,0.024880,0.000619,-2.219704e-02,0.054460,0.002966,-0.000095,0.018283,0.000334,2.121143,1.372578,1.883970,3
6,7,107.666016,138107,1290.719626,0.246077,0.299126,0.089476,0.386659,0.275599,0.075955,0.235532,0.166907,0

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
plt.style.use('dark_background')

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM

In [ ]:
training_set = df_train.iloc[:,1:54].values

In [ ]:
import random
random.shuffle(training_set)
X_test = training_set[-100:]
training_set = training_set[:-100]
print(training_set)
# print(X_test)
X_test_final = []
for i in range (60,100):
  X_test_final.append(X_test[i-60:i,0])

[[1.17453835e+02 1.43490000e+05 1.25868421e+03 ... 8.83668200e-03
  1.76587755e+00 1.09241126e+00]
 [1.29199219e+02 1.64643000e+05 1.27630233e+03 ... 7.39908000e-04
  8.46715646e+00 4.64921988e+00]
 [1.17453835e+02 1.43490000e+05 1.25868421e+03 ... 8.83668200e-03
  1.76587755e+00 1.09241126e+00]
 ...
 [9.57031250e+01 1.19832000e+05 1.31683516e+03 ... 5.66691000e-04
  4.09251701e+00 1.10483743e+00]
 [1.72265625e+02 2.19012000e+05 1.32734546e+03 ... 1.03351400e-03
  4.34561451e+00 1.18132931e+00]
 [1.43554688e+02 1.70821000e+05 1.24686861e+03 ... 1.66982000e-04
  2.10372789e+00 1.34151065e+00]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
print(training_set_scaled)

[[0.35064935 0.38152352 0.33738388 ... 0.38082447 0.01568704 0.01774758]
 [0.42857143 0.4808609  0.35541754 ... 0.0314006  0.2403659  0.20791217]
 [0.35064935 0.38152352 0.33738388 ... 0.38082447 0.01568704 0.01774758]
 ...
 [0.20634921 0.27042232 0.39690641 ... 0.02392526 0.09369404 0.01841195]
 [0.71428571 0.73618514 0.40766459 ... 0.04407145 0.10217984 0.02250158]
 [0.52380952 0.50987363 0.3252896  ... 0.00667545 0.0270144  0.03106567]]


In [ ]:
X_train = []
y_train = []
for i in range(60, 1272):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
    
    # X_train.append(training_set[i-60:i, 0])
    # y_train.append(training_set[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_test_final = np.array(X_test_final)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_final = np.reshape(X_test_final, (X_test_final.shape[0],X_test_final.shape[1],1))
x_val = X_train[-100:]
y_val = y_train[-100:]
X_train = X_train[:-100]
y_train = y_train[:-100]
# print(X_train)

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 64, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 64, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 64))
# regressor.add(Dropout(0.2))

regressor.add(Dense(units = 64, activation='relu'))
regressor.add(Dropout(0.1))
regressor.add(Dense(64, activation='relu'))


# regressor.add(Dense(32,activation='softmax'))

regressor.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['accuracy'])

regressor.fit(X_train, y_train, epochs = 100, validation_data=(x_val,y_val), batch_size = 32)

Train on 1112 samples, validate on 100 samples
Epoch 1/100
1112/1112 [==============================] - 6s 5ms/step - loss: 0.6933 - accuracy: 0.0171 - val_loss: 0.6931 - val_accuracy: 0.3500
Epoch 2/100
1112/1112 [==============================] - 4s 4ms/step - loss: 0.6931 - accuracy: 0.3867 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 3/100
1112/1112 [==============================] - 4s 4ms/step - loss: 0.6931 - accuracy: 0.7050 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 4/100
1112/1112 [==============================] - 4s 4ms/step - loss: 0.6931 - accuracy: 0.8219 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 5/100
1112/1112 [==============================] - 4s 4ms/step - loss: 0.6931 - accuracy: 0.8759 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 6/100
1112/1112 [==============================] - 4s 4ms/step - loss: 0.6931 - accuracy: 0.9074 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 7/100
1112/1112 [==============================] - 4s 4ms/step - loss: 0.6931

In [ ]:
predict_output = regressor.predict(X_test_final)

In [ ]:
print(predict_output)

[[0.         0.16574945 0.         ... 0.         0.02686475 0.04035877]
 [0.         0.16595042 0.         ... 0.         0.02677922 0.04036916]
 [0.         0.16570547 0.         ... 0.         0.02695834 0.04071589]
 ...
 [0.         0.16499    0.         ... 0.         0.02707372 0.04000176]
 [0.         0.16517092 0.         ... 0.         0.02697506 0.03980707]
 [0.         0.16547696 0.         ... 0.         0.02681224 0.03998208]]
